In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time, json, random

# ───────────────
# 브라우저 옵션 설정
# ───────────────
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)

# ───────────────
# 카테고리 진입 (칼슘)
# ───────────────
url = "https://search.shopping.naver.com/ns/category/10002487?sort=PURCHASE"
driver.get(url)
time.sleep(random.uniform(3, 5))

# ───────────────
# 스크롤 다운
# ───────────────
SCROLL_COUNT = 2
for _ in range(SCROLL_COUNT):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(2, 4))

# 최상단으로 이동 (선택사항)
driver.execute_script("window.scrollTo(0, 0);")
time.sleep(random.uniform(2, 4))

# ───────────────
# 상품 URL 수집
# ───────────────
product_urls = []
soup = BeautifulSoup(driver.page_source, "html.parser")
cards = soup.select("a.basicProductCard_link__urzND")

for card in cards:
    href = card.get("href")
    full_url = href if href.startswith("http") else "https://smartstore.naver.com" + href
    if full_url not in product_urls:
        product_urls.append(full_url)

# 중복 제거 및 상위 100개 제한
product_urls = list(dict.fromkeys(product_urls))[:100]
print(f"✅ 수집된 상품 URL 수: {len(product_urls)}")

# ───────────────
# 상세페이지 순회
# ───────────────
results = []

for idx, url in enumerate(product_urls):
    driver.get(url)
    time.sleep(random.uniform(6, 10))

    if idx % 10 == 0 and idx != 0:
        print(f"🛑 {idx}개 수집 완료 - 추가 대기 중...")
        time.sleep(10 + random.uniform(3, 6))

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # ✅ 제품명 추출 (fallback 포함)
    try:
        # 먼저 BeautifulSoup로 시도
        title_tag = soup.select_one("div._1eddO7u4UC > h3") or soup.select_one("h3._22kNQuEXmb")
        if title_tag:
            name = title_tag.get_text(strip=True)
        else:
            # fallback: Selenium 직접 접근
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "h3._22kNQuEXmb"))
            )
            name = driver.find_element(By.CSS_SELECTOR, "h3._22kNQuEXmb").text.strip()
    except:
        name = "N/A"

    # ✅ 가격 추출 (기존 방식 유지)
    price = "N/A"
    price_tags = soup.select("span._1LY7DqCnwR")
    if len(price_tags) >= 2:
        price = price_tags[1].get_text(strip=True).replace(",", "")

    # ✅ 이미지 추출
    image_tag = soup.select_one("img.bd_2DO68")
    image = image_tag["src"] if image_tag else "N/A"

    # 저장
    results.append({
        "name": name,
        "image": image,
        "price": price,
        "url": url,
        "nutrition": {}
    })

    print(f"[{idx+1}/{len(product_urls)}] ✅ {name} 저장 완료")

# 종료
driver.quit()

# ───────────────
# JSON 저장
# ───────────────
with open("calcium_50.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print("🎉 calcium_50.json 저장 완료!")


✅ 수집된 상품 URL 수: 50
[1/50] ✅ [총 9개월분] GNM 칼슘 마그네슘 아연 비타민D 90정 x 3병 [원산지:상세설명에 표시] 저장 완료
[2/50] ✅ 쁘띠앤 국민 리포좀 칼마디 칼슘 마그네슘 비타민D 영양제 임산부 수유부 청소년 뼈건강 칼슘마그네슘비타민D 2개월 [원산지:상세설명에 표시] 저장 완료
[3/50] ✅ 종근당 칼슘 마그네슘 아연 비타민D 3개월 칼마디 골다공증 뼈 치아 임산부 어린이 마칼디 [원산지:상세설명에 표시] 저장 완료
[4/50] ✅ 세비톨 칼슘 마그네슘 비타민D 비타민K2 임산부 수유부 칼마디 칼슘영양제 120정, 1개 [원산지:상세설명에 표시] 저장 완료
[5/50] ✅ 닥터브라이언 칼마디 칼슘 마그네슘 비타민D 아연 90정, 1개 [원산지:미국] 저장 완료
[6/50] ✅ 종근당 뼈에좋은 칼슘 마그네슘 비타민D 아연 영양제 6개월 칼마디 임산부 마칼디 [원산지:상세설명에 표시] 저장 완료
[7/50] ✅ 뉴트리모어 프리미엄 임산부 수유부 칼마디 칼슘 마그네슘 비타민D3 영양제 60정, 4개 [원산지:상세설명에 표시] 저장 완료
[8/50] ✅ 칼마디 K2 어바인랩 WPE 칼슘 마그네슘 비타민D 비타민K2 임산부 60캡슐, 1개 [원산지:국산(충청북도 제천시)] 저장 완료
[9/50] ✅ 종근당 뼈건강 프로젝트365 칼마디아K 1,100mgx180정 2+1 총3박스 9개월 칼슘 [원산지:상세설명에 표시] 저장 완료
[10/50] ✅ 덴티에센셜 3개월분 뼈 치아 칼슘 영양제 / 잇몸 과학 치 약 포함 좋은 세트 [원산지:상세설명에 표시] 저장 완료
🛑 10개 수집 완료 - 추가 대기 중...
[11/50] ✅ 차일드라이프 액상칼슘마그네슘 1병 비타민D 아연 돌 아기 유아 어린이 키즈 키성장기 영양제 [원산지:미국] 저장 완료
[12/50] ✅ 차일드라이프 액상칼슘마그네슘 어린이 칼마디 돌 아기 비타민D 아연 키즈 키성장기 영양제 [원산지:미국] 저장 완료
[13/50] ✅ 종근당 칼슘 마그네슘 비타민D 망간 칼마디 

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time, json, random

# 브라우저 옵션 설정 (똥꼬쇼)
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)

# ───────────────
# 원하는 카테고리 카테고리 판매순 (비타민, 마그네슘, 칼슘, 철분, 아연)
url = "https://search.shopping.naver.com/ns/category/10002487?sort=PURCHASE" # 칼슘
driver.get(url)
time.sleep(random.uniform(3, 5))

# 스크롤
SCROLL_COUNT = 2
for _ in range(SCROLL_COUNT):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(4, 7))

# 최상단으로 복귀
driver.execute_script("window.scrollTo(0, 0);")
time.sleep(random.uniform(2,4))

# ───────────────
# 상품 URL 수집
# ───────────────
product_urls = []
soup = BeautifulSoup(driver.page_source, "html.parser")
cards = soup.select("a.basicProductCard_link__urzND")

for card in cards:
    href = card.get("href")
    full_url = href if href.startswith("http") else "https://smartstore.naver.com" + href
    if full_url not in product_urls:
        product_urls.append(full_url)

product_urls = product_urls[:100]
print(f"✅ 수집된 상품 URL 수: {len(product_urls)}")


# 상품 URL 정제
# product_urls = []
# for card in cards:
#     href = card.get("href")
#     if href.startswith("http"):
#         product_urls.append(href)
#     else:
#         product_urls.append("https://smartstore.naver.com" + href)

# 중복 제거 ( 선택 ??)
product_urls = list(dict.fromkeys(product_urls))

# 상위 100개
product_urls = product_urls[:100]

# Let's do it again

results = []

for idx, url in enumerate(product_urls):
    driver.get(url)
    time.sleep(random.uniform(6, 10))  # Unifom

    if idx % 10 == 0 and idx != 0:
        print(f"🛑 {idx}개 수집 완료 - 추가 대기 중...")
        time.sleep(10 + random.uniform(3, 6))  # ✅ Avoid Bot detec

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 제품명
    title_tag = (
    soup.select_one("div._1eddO7u4UC > h3") or
    soup.select_one("h3._22kNQuEXmb")  # 여기 추가!
)
    name = title_tag.get_text(strip=True) if title_tag else "N/A"

    # 가격 (그 원 가격 말고 할인된 가격으로)
    price = "N/A"
    price_tags = soup.select("span._1LY7DqCnwR")
    if len(price_tags) >= 2:
        price = price_tags[1].get_text(strip=True).replace(",", "")

    # 이미지
    image_tag = soup.select_one("img.bd_2DO68")
    image = image_tag["src"] if image_tag else "N/A"

    # 저장
    results.append({
        "name": name,
        "image": image,
        "price": price,
        "url": url,
        "nutrition": {}
    })

    print(f"[{idx+1}/{len(product_urls)}] ✅ {name} 저장 완료")

driver.quit()

# JSON 저장
with open("calcium_100.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print("🎉 magnesium_100.json 저장 완료!")

✅ 수집된 상품 URL 수: 50
[1/50] ✅ [총 9개월분] GNM 칼슘 마그네슘 아연 비타민D 90정 x 3병 [원산지:상세설명에 표시] 저장 완료
[2/50] ✅ 쁘띠앤 국민 리포좀 칼마디 칼슘 마그네슘 비타민D 영양제 임산부 수유부 청소년 뼈건강 칼슘마그네슘비타민D 2개월 [원산지:상세설명에 표시] 저장 완료
[3/50] ✅ 종근당 칼슘 마그네슘 아연 비타민D 3개월 칼마디 골다공증 뼈 치아 임산부 어린이 마칼디 [원산지:상세설명에 표시] 저장 완료
[4/50] ✅ 세비톨 칼슘 마그네슘 비타민D 비타민K2 임산부 수유부 칼마디 칼슘영양제 120정, 1개 [원산지:상세설명에 표시] 저장 완료
[5/50] ✅ 닥터브라이언 칼마디 칼슘 마그네슘 비타민D 아연 90정, 1개 [원산지:미국] 저장 완료
[6/50] ✅ 종근당 뼈에좋은 칼슘 마그네슘 비타민D 아연 영양제 6개월 칼마디 임산부 마칼디 [원산지:상세설명에 표시] 저장 완료
[7/50] ✅ 뉴트리모어 프리미엄 임산부 수유부 칼마디 칼슘 마그네슘 비타민D3 영양제 60정, 4개 [원산지:상세설명에 표시] 저장 완료
[8/50] ✅ 칼마디 K2 어바인랩 WPE 칼슘 마그네슘 비타민D 비타민K2 임산부 60캡슐, 1개 [원산지:국산(충청북도 제천시)] 저장 완료
[9/50] ✅ 종근당 뼈건강 프로젝트365 칼마디아K 1,100mgx180정 2+1 총3박스 9개월 칼슘 [원산지:상세설명에 표시] 저장 완료
[10/50] ✅ 덴티에센셜 3개월분 뼈 치아 칼슘 영양제 / 잇몸 과학 치 약 포함 좋은 세트 [원산지:상세설명에 표시] 저장 완료
🛑 10개 수집 완료 - 추가 대기 중...
[11/50] ✅ 차일드라이프 액상칼슘마그네슘 1병 비타민D 아연 돌 아기 유아 어린이 키즈 키성장기 영양제 [원산지:미국] 저장 완료
[12/50] ✅ 차일드라이프 액상칼슘마그네슘 어린이 칼마디 돌 아기 비타민D 아연 키즈 키성장기 영양제 [원산지:미국] 저장 완료
[13/50] ✅ 종근당 칼슘 마그네슘 비타민D 망간 칼마디 

### 계속 테스트용

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time, json, random

# ───────────────
# 브라우저 옵션 설정
# ───────────────
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)

# ───────────────
# 비타민 카테고리 접속 (판매순 정렬)
# ───────────────
url = "https://search.shopping.naver.com/ns/category/10000315?sort=PURCHASE"
driver.get(url)
time.sleep(random.uniform(3, 5))

# ───────────────
# 스크롤 자동 내리기 (약 100개 상품 로드)
# ───────────────
SCROLL_COUNT = 10
for _ in range(SCROLL_COUNT):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(2, 4))  # 랜덤 대기시간

# 최상단으로 복귀
driver.execute_script("window.scrollTo(0, 0);")
time.sleep(random.uniform(2,4))

# ───────────────
# 상품 URL 수집
# ───────────────
product_urls = []
soup = BeautifulSoup(driver.page_source, "html.parser")
cards = soup.select("a.basicProductCard_link__urzND")

for card in cards:
    href = card.get("href")
    full_url = href if href.startswith("http") else "https://smartstore.naver.com" + href
    if full_url not in product_urls:
        product_urls.append(full_url)

product_urls = product_urls[:100]
print(f"✅ 수집된 상품 URL 수: {len(product_urls)}")

# ───────────────
# 상품 카드 수집
# ───────────────
soup = BeautifulSoup(driver.page_source, "html.parser")
cards = soup.select("a.basicProductCard_link__urzND")

# ───────────────
# 상품 URL 정제
# ───────────────
product_urls = []
for card in cards:
    href = card.get("href")
    if href.startswith("http"):
        product_urls.append(href)
    else:
        product_urls.append("https://smartstore.naver.com" + href)

# 중복 제거
product_urls = list(dict.fromkeys(product_urls))

# 상위 100개만 사용
product_urls = product_urls[:100]

# ───────────────
# 각 상품 상세페이지에서 정보 수집
# ───────────────
results = []

for idx, url in enumerate(product_urls):
    driver.get(url)
    time.sleep(random.uniform(6, 10))  # ✅ 기본 대기시간 증가

    if idx % 10 == 0 and idx != 0:
        print(f"🛑 {idx}개 수집 완료 - 추가 대기 중...")
        time.sleep(10 + random.uniform(3, 6))  # ✅ 10개 단위 추가 대기

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 제품명
    title_tag = soup.select_one("div._1eddO7u4UC > h3")
    name = title_tag.get_text(strip=True) if title_tag else "N/A"

    # 가격 (두 번째 가격: 할인된 가격)
    price = "N/A"
    price_tags = soup.select("span._1LY7DqCnwR")
    if len(price_tags) >= 2:
        price = price_tags[1].get_text(strip=True).replace(",", "")

    # 이미지
    image_tag = soup.select_one("img.bd_2DO68")
    image = image_tag["src"] if image_tag else "N/A"

    # 저장
    results.append({
        "name": name,
        "image": image,
        "price": price,
        "url": url,
        "nutrition": {}
    })

    print(f"[{idx+1}/{len(product_urls)}] ✅ {name} 저장 완료")

driver.quit()

# ───────────────
# JSON 저장
# ───────────────
with open("vitamin_100.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print("🎉 vitamin_100.json 저장 완료!")

✅ 수집된 상품 URL 수: 50
[1/50] ✅ 고려은단 비타민C 1000 600정 X 1개 (20개월분) 저장 완료
[2/50] ✅ 고려은단 멀티비타민 올인원 60정, 3개 저장 완료
[3/50] ✅ 고려은단 메가도스C 3000 영국산 비타민C 100포, 1개 저장 완료
[4/50] ✅ 고려은단 비타민C 1000 이지+비타민D 180정, 2개 (6개월분) 저장 완료
[5/50] ✅ 고려은단 메가도스3000 고함량 비타민씨 고용량 분말 가루 스틱 영국산비타민C 2개월 저장 완료
[6/50] ✅ 비타민c 종근당 1000mg 600정 바이탈프로그램 씨 선물 용 항산화 아스코르빈산 저장 완료
[7/50] ✅ 압타민C 저장 완료
[8/50] ✅ 멀티비타민 오쏘몰 이뮨 비타민 30일분 1박스 + 1일분 + 쇼핑백 증정 저장 완료
[9/50] ✅ 고려은단 비타민C 1000 영국산 고함량 비타민씨 600정 저장 완료
[10/50] ✅ [조정석 멀티비타민+밀크씨슬+루테인+알티지오메가3]GNM 올인원 뉴트리션 멀티비타민 1박스 저장 완료
🛑 10개 수집 완료 - 추가 대기 중...
[11/50] ✅ [네이버 단독]고려은단 멀티비타민 이뮨샷 액상비타민 30병, 1개 저장 완료
[12/50] ✅ 속편한 고려은단 비타민C 1000 중성비타민C 120정, 2개 (8개월분) 저장 완료
[13/50] ✅ 고려은단 메가도스C 2000 영국산 비타민C 100포, 1개 저장 완료
[14/50] ✅ 피에이치365 아기 비타민D 아연 베이비 돌 유아 어린이 키즈 영양제 저장 완료
[15/50] ✅ 프리솔라 비타민D 2000IU 2개월분 성인 어린이 임산부 저장 완료
[16/50] ✅ 바노 이왕재 박사 메가 비타민C 2000mg 고함량 영국산 분말 비타민씨 90포, 8개 저장 완료
[17/50] ✅ 고려은단 메가도스D 비타민D3 4000IU 120정, 3개 (1년분) 저장 완료
[18/50] ✅ 종근당 고함량 비타민B 비타민비 군 영양제 활성 수용성 컴플렉스 B12 B1 B3 B6 저장 완료
[

### 현재까진 best, 근데 스크롤 시작점부터 수집한다는 단점 존재

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time, json, random

# ───────────────
# 브라우저 옵션 설정
# ───────────────
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)

# ───────────────
# 비타민 카테고리 접속 (판매순 정렬)
# ───────────────
url = "https://search.shopping.naver.com/ns/category/10000315?sort=PURCHASE"
driver.get(url)
time.sleep(random.uniform(3, 5))

# ───────────────
# 스크롤 자동 내리기 (약 100개 상품 로드)
# ───────────────
SCROLL_COUNT = 5
for _ in range(SCROLL_COUNT):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(2, 4))  # 랜덤 대기시간

# ───────────────
# 상품 카드 수집
# ───────────────
soup = BeautifulSoup(driver.page_source, "html.parser")
cards = soup.select("a.basicProductCard_link__urzND")

# ───────────────
# 상품 URL 정제
# ───────────────
product_urls = []
for card in cards:
    href = card.get("href")
    if href.startswith("http"):
        product_urls.append(href)
    else:
        product_urls.append("https://smartstore.naver.com" + href)

# 중복 제거
product_urls = list(dict.fromkeys(product_urls))

# 상위 100개만 사용
product_urls = product_urls[:100]

# ───────────────
# 각 상품 상세페이지에서 정보 수집
# ───────────────
results = []

for idx, url in enumerate(product_urls):
    driver.get(url)
    time.sleep(random.uniform(6, 10))  # ✅ 기본 대기시간 증가

    if idx % 10 == 0 and idx != 0:
        print(f"🛑 {idx}개 수집 완료 - 추가 대기 중...")
        time.sleep(10 + random.uniform(3, 6))  # ✅ 10개 단위 추가 대기

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 제품명
    title_tag = soup.select_one("div._1eddO7u4UC > h3")
    name = title_tag.get_text(strip=True) if title_tag else "N/A"

    # 가격 (두 번째 가격: 할인된 가격)
    price = "N/A"
    price_tags = soup.select("span._1LY7DqCnwR")
    if len(price_tags) >= 2:
        price = price_tags[1].get_text(strip=True).replace(",", "")

    # 이미지
    image_tag = soup.select_one("img.bd_2DO68")
    image = image_tag["src"] if image_tag else "N/A"

    # 저장
    results.append({
        "name": name,
        "image": image,
        "price": price,
        "url": url,
        "nutrition": {}
    })

    print(f"[{idx+1}/{len(product_urls)}] ✅ {name} 저장 완료")

driver.quit()

# ───────────────
# JSON 저장
# ───────────────
with open("vitamin_100.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=4, ensure_ascii=False)

print("🎉 vitamin_100.json 저장 완료!")

📦 상품 URL 수집 중...
✅ 현재 수집된 상품 수: 50
✅ 현재 수집된 상품 수: 100
[1] ❌ 제품명 로딩 실패


InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00007FF649854C25+3179557]
	(No symbol) [0x00007FF6494B88A0]
	(No symbol) [0x00007FF649348FFC]
	(No symbol) [0x00007FF64938F8DF]
	(No symbol) [0x00007FF6493C7AF2]
	(No symbol) [0x00007FF6493C244E]
	(No symbol) [0x00007FF6493C14F9]
	(No symbol) [0x00007FF6493155E5]
	GetHandleVerifier [0x00007FF6498B67CD+3579853]
	GetHandleVerifier [0x00007FF6498CD1D2+3672530]
	GetHandleVerifier [0x00007FF6498C2153+3627347]
	GetHandleVerifier [0x00007FF64962092A+868650]
	(No symbol) [0x00007FF6494C2FFF]
	(No symbol) [0x00007FF6493141FF]
	GetHandleVerifier [0x00007FF64993FA28+4141608]
	BaseThreadInitThunk [0x00007FFE85BF7374+20]
	RtlUserThreadStart [0x00007FFE87A9CC91+33]
